In [142]:
import pandas as pd
import glob as glob
import numpy as np

In [143]:
TFs_tbl = pd.read_csv("../soto_analysis/outputs/all_TFs_table_proteins.txt", sep = "\t")
TFs_tbl

,Unnamed: 0,1,2,uniprotID,ENSG,ENST,DBD_coords,AD_coords,RD_coords,Bif_coords,length
0,0,NaN,NaN,A6NJG6,NaN,ENST00000334384,79-135,142-315,NaN,NaN,1
1,1,NaN,NaN,A8MTJ6,NaN,ENST00000428390,145-234,369-420,NaN,NaN,1
2,2,NaN,NaN,A8MYZ6,NaN,ENST00000641094,89-178,382-491,NaN,NaN,1
3,3,NaN,NaN,A8MZ59,NaN,ENST00000640845,31-65,92-181,NaN,NaN,1
4,4,NaN,NaN,O00321,NaN,ENST00000403402,242-321,1-162,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
441,441,NaN,NaN,Q9Y5R6,NaN,ENST00000382276,72-118,172-251,NaN,NaN,1
442,442,NaN,NaN,Q9Y5W3,NaN,ENST00000248071,"272-296,302-326,332-354",1-89,NaN,NaN,1
443,443,NaN,NaN,Q9Y692,NaN,ENST00000294409,89-165,422-573,NaN,NaN,1
444,444,NaN,NaN,Q9Y6Q9,NaN,ENST00000371998,31-83,621-1424,NaN,NaN,1


In [144]:
tf_domain_beds = glob.glob("../soto_analysis/outputs/mutations/domains_bed_format/*")
tf_domain_beds[:3]

['../soto_analysis/outputs/mutations/domains_bed_format/Q5XKR4',
 '../soto_analysis/outputs/mutations/domains_bed_format/Q8N587',
 '../soto_analysis/outputs/mutations/domains_bed_format/P37231']

In [145]:
#https://www.geeksforgeeks.org/python-make-a-list-of-intervals-with-sequential-numbers/
def interval_extract(input_list):
    input_list = sorted(set(input_list))
    range_start = previous_number = input_list[0]
 
    for number in input_list[1:]:
        if number == previous_number + 1:
            previous_number = number
        else:
            yield [range_start, previous_number]
            range_start = previous_number = number
    yield [range_start, previous_number]

In [146]:
def convert_rec(x):
    if isinstance(x, list):
        return list(map(convert_rec, x))
    else:
        return int(x)

In [156]:
def explode_df(input_df, strand):
    df = input_df.copy(deep = True)
    df["all_pos"] = [np.arange(start, end ) for start, end in zip(df[1], df[2])]
    df_exploded = df.explode("all_pos")
    if strand == -1:
        df_exploded = df_exploded.sort_values(by = "all_pos", ascending = False)
    else:
        df_exploded = df_exploded.sort_values(by = "all_pos", ascending = True)
    
    df_exploded = df_exploded.reset_index(drop = True)
    return df_exploded
    
for bed in tf_domain_beds:
    #print(bed)

    # REading in exising bed file of domain coordinates
    domain_bed_df = pd.read_csv(bed, sep = "\t", header = None)
    
    uniprotID = bed.split("/")[-1]
    row = TFs_tbl[TFs_tbl["uniprotID"] == uniprotID]
    #display(row)

    # Comparing to CDS
    ENST = row["ENST"].iloc[0]
    cds_bed_df = pd.read_csv("../soto_analysis/outputs/mutations/cds_bed_format/" + ENST, sep = "\t", header = None)
    # cds_bed_df[1] += 1
    # cds_bed_df[2] += 1
    strand = cds_bed_df[4].iloc[0]
    #print(strand)
    #print(ENST)
    # Adding one row per position and calc corresp protein position
    cds_bed_df_exploded = explode_df(cds_bed_df, strand)
    cds_bed_df_exploded["prot_pos"] = 1 + cds_bed_df_exploded.index // 3

    domain_bed_df_exploded = explode_df(domain_bed_df, strand)

    overlap = pd.merge(cds_bed_df_exploded[["all_pos", "prot_pos"]], domain_bed_df_exploded[["all_pos", 3]], on = "all_pos")

    for domain in ["AD", "DBD"]:
        domain_positions = overlap[overlap[3] == domain]
        expec = row[domain + "_coords"].iloc[0]

        if len(domain_positions) > 0:
            # Checking what the observed intervals of protein positiosn are where there is a matchign AD doman pos
            obs = list(interval_extract(domain_positions["prot_pos"]))
            expec = row[domain + "_coords"].iloc[0]
            expec = expec.split(",")
            expec = [_.split("-") for _ in expec]
            expec = convert_rec(expec)
            expec.sort()
            if (obs != expec):
                print("check " + uniprotID + " " + domain)
                display("expec: " + str(expec))
                display("obs: " + str(obs))
        else:
            if expec != "NA-NA":
                print("check " + uniprotID + " " + domain)
                display("expec: " + str(expec))
                display("obs: " + str(obs))        


    # print("done checking " + uniprotID)
    # print()


    

        
    

check P10070 AD


'expec: [[659, 1197], [1198, 1586]]'

'obs: [[659, 1586]]'

check Q14872 AD


'expec: [[328, 509], [510, 624]]'

...

'expec: [[191, 421], [422, 444]]'

'obs: [[191, 444]]'

check Q13887 AD


'expec: [[1, 238], [239, 372]]'

'obs: [[1, 372]]'